<a href="https://colab.research.google.com/github/DOI03/DOI03/blob/main/CV_AL_test4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.   optimizer로 RAdam 사용
2.   lr = 0.0001
3.   get_cosine_schedule_with_warmup 스케줄러 사용





In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install git+https://github.com/LiyuanLucasLiu/RAdam.git
!pip install wandb

  Cloning https://github.com/LiyuanLucasLiu/RAdam.git to /tmp/pip-req-build-48agp0_z
  Running command git clone --filter=blob:none --quiet https://github.com/LiyuanLucasLiu/RAdam.git /tmp/pip-req-build-48agp0_z
  Resolved https://github.com/LiyuanLucasLiu/RAdam.git to commit d9fd30a337894c4003768561d45e8730dbd41333
  Preparing metadata (setup.py) ... done
  Created wheel for RAdam: filename=RAdam-0.0.1-py3-none-any.whl size=7098 sha256=f95a8d60c457e4ecf581ffe6f0de6b20a600583307e3a89dd37d90f42c50cda3
  Stored in directory: /tmp/pip-ephem-wheel-cache-mg7e1cvn/wheels/27/dd/f4/a154a2dd1d807820314f09ba2fa30f13f96a0d2830e8bab05e
Successfully built RAdam


In [3]:
import os
import time

import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import wandb

# radam 사용
from radam import RAdam

# 스케줄러 사용
from transformers import get_cosine_schedule_with_warmup

from PIL import Image
#내가 추가
import re

# wandb 설정 (오프라인 모드로 실행)
wandb.init(project="CUB_Transfer_Learning", name="RAdam and get_cosine_schedule_with_warmup")

### GPU Setting ###
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


cuda


In [4]:
### Custom Dataset ###
class CUB2011(Dataset):
  def __init__(self, transform, mode='train'):
    self.transform = transform
    self.mode = mode

    if self.mode == 'train':
      self.image_folder = os.listdir('/content/gdrive/MyDrive/CUB_200_2011_repackage_class50/datasets/train')
    elif self.mode == 'valid':
      self.image_folder = os.listdir('/content/gdrive/MyDrive/CUB_200_2011_repackage_class50/datasets/valid')
    elif self.mode == 'test':
      self.image_folder = os.listdir('/content/gdrive/MyDrive/CUB_200_2011_repackage_class50/datasets/test')

  def __len__(self):
    return len(self.image_folder)

  def __getitem__(self, idx):
    img_path = self.image_folder[idx]
    img = Image.open(os.path.join('/content/gdrive/MyDrive/CUB_200_2011_repackage_class50/datasets', self.mode, img_path)).convert('RGB')
    img = self.transform(img)
    label = img_path.split('_')[-1].split('.')[0]
    label = re.sub(r'\([^)]*\)', '', label)
    label = int(label)
    return (img, label)

In [5]:
### Data Preprocessing ###

transforms_train = transforms.Compose([transforms.Resize((448,448)),
                                       transforms.ToTensor()])
transforms_valtest = transforms.Compose([transforms.Resize((448,448)),
                                         transforms.ToTensor()])

BATCH_SIZE = 32
train_set = CUB2011(mode='train',
                    transform=transforms_train)
# transforms_train을 transforms_valtest로 변경
val_set = CUB2011(mode='valid',
                  transform=transforms_valtest)
test_set = CUB2011(mode='test',
                  transform=transforms_valtest)

print('Num of each dataset: ',len(train_set),len(val_set),len(test_set))

train_loader = DataLoader(train_set,batch_size=BATCH_SIZE,shuffle=True)
val_loader = DataLoader(val_set,batch_size=BATCH_SIZE,shuffle=False)
test_loader = DataLoader(test_set,batch_size=BATCH_SIZE,shuffle=False)

print("Loaded dataloader")

### Model / Optimizer ###
EPOCH = 25
lr = 0.0001

model = models.resnet18(pretrained=True)

### Tranfer Learning ###
num_features = model.fc.in_features
model.fc = nn.Linear(num_features,50)
model.to(DEVICE)

optimizer = optim.RAdam(model.parameters(),lr=lr)

# 스케줄러 초기화 (get_cosine_schedule_with_warmup 사용)
total_steps = len(train_loader) * EPOCH
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=total_steps // 10, num_training_steps=total_steps)

print("Created a learning model and optimizer")

Num of each dataset:  2360 296 298
Loaded dataloader


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 99.0MB/s]


Created a learning model and optimizer


In [6]:
### Train/Evaluation ###
def train(model,train_loader,optimizer,epoch):
  model.train()
  for i,(image,target) in enumerate(train_loader):
    image,target = image.to(DEVICE),target.to(DEVICE)
    output = model(image)
    optimizer.zero_grad()
    train_loss = F.cross_entropy(output,target).to(DEVICE)

    train_loss.backward()
    optimizer.step()

    # 스케줄러 업데이트
    scheduler.step()

    if i%10 ==0:
      print(
          f'Train Epoch: {epoch} [{i}/{len(train_loader)}]\tloss: {train_loss.item():6f}')

  return train_loss

def evaluate(model,val_loader):
  model.eval()
  eval_loss = 0
  correct = 0
  with torch.no_grad():
    for i,(image,target) in enumerate(val_loader):
      image,target = image.to(DEVICE),target.to(DEVICE)
      output = model(image)

      eval_loss += F.cross_entropy(output,target, reduction='sum').item()
      pred = output.max(1,keepdim=True)[1]
      correct += pred.eq(target.view_as(pred)).sum().item()

  eval_loss /= len(val_loader.dataset)
  eval_accuracy = 100*correct / len(val_loader.dataset)
  return eval_loss,eval_accuracy

In [7]:
### Main ###
start = time.time()
best = 0
for epoch in range(EPOCH):
    train_loss = train(model, train_loader, optimizer, epoch)
    val_loss, val_accuracy = evaluate(model, val_loader)

    # wandb 로그 기록
    wandb.log({
        'Epoch': epoch,
        'Train Loss': train_loss,
        'Validation Loss': val_loss,
        'Validation Accuracy': val_accuracy
    })

    # Save best model
    if val_accuracy > best:
        best = val_accuracy
        torch.save(model.state_dict(), "./best_model.pth")

    print(f"[{epoch}] Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}%")

# Test result
test_loss, test_accuracy = evaluate(model, test_loader)
print(f'[FINAL] Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}%')

# wandb 테스트 결과 로그 기록
wandb.log({
    'Test Loss': test_loss,
    'Test Accuracy': test_accuracy
})

end = time.time()
elapsed_time = end - start

print("Best Accuracy: ", best)
print(f"Elapsed Time: {int(elapsed_time / 3600)}h, {int(elapsed_time / 60)}m, {int(elapsed_time % 60)}s")

wandb.finish()

Train Epoch: 0 [0/74]	loss: 4.069471
Train Epoch: 0 [10/74]	loss: 4.175213
Train Epoch: 0 [20/74]	loss: 4.003531
Train Epoch: 0 [30/74]	loss: 3.816018
Train Epoch: 0 [40/74]	loss: 3.623078
Train Epoch: 0 [50/74]	loss: 3.364537
Train Epoch: 0 [60/74]	loss: 3.014391
Train Epoch: 0 [70/74]	loss: 3.006881
[0] Validation Loss: 2.6544, Accuracy: 43.5811%
Train Epoch: 1 [0/74]	loss: 2.523283
Train Epoch: 1 [10/74]	loss: 2.284463
Train Epoch: 1 [20/74]	loss: 2.114076
Train Epoch: 1 [30/74]	loss: 1.892113
Train Epoch: 1 [40/74]	loss: 1.720813
Train Epoch: 1 [50/74]	loss: 1.669986
Train Epoch: 1 [60/74]	loss: 1.235622
Train Epoch: 1 [70/74]	loss: 1.303895
[1] Validation Loss: 1.5208, Accuracy: 67.2297%
Train Epoch: 2 [0/74]	loss: 0.936924
Train Epoch: 2 [10/74]	loss: 0.743878
Train Epoch: 2 [20/74]	loss: 0.688905
Train Epoch: 2 [30/74]	loss: 0.552266
Train Epoch: 2 [40/74]	loss: 0.782589
Train Epoch: 2 [50/74]	loss: 1.127929
Train Epoch: 2 [60/74]	loss: 1.004723
Train Epoch: 2 [70/74]	loss: 0.87

Epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
Test Accuracy,▁
Test Loss,▁
Train Loss,█▃▃▄▄▄▄▆▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▄▆▃▁▁▂▂▄▅▅▄▆▆▇▇▇█████████
Validation Loss,▃▂▃▄▄█▅▂▂▂▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,24
Test Accuracy,79.19463
Test Loss,0.81488
Train Loss,0.00131
Validation Accuracy,82.43243
